In [ ]:
!pip install wandb

In [ ]:
!pip install retina-face

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D
import wandb, cv2
import numpy as np
from wandb.keras import WandbCallback
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from retinaface import RetinaFace

In [ ]:
tf.__version__

'2.6.0'

### 🔸Generate Dataset

In [ ]:
wandb.init(project='Face_Recognition')

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
folder_address = '/content/drive/MyDrive/Datasets/7-7 dataset/'

batch_size = 16
width = height = 224

config = wandb.config
config.learning_rate = 0.001

In [ ]:
data_generator = ImageDataGenerator(
    rescale = 1. / 255,
    validation_split = 0.1,
    horizontal_flip = True
)

train_data = data_generator.flow_from_directory(folder_address,
                                                    target_size = (width, height),
                                                    class_mode = 'categorical',
                                                    batch_size = batch_size,
                                                    shuffle = True,
                                                    subset ='training')

val_data = data_generator.flow_from_directory(folder_address,
                                                    target_size = (width, height),
                                                    class_mode = 'categorical',
                                                    batch_size = batch_size,
                                                    shuffle = False,
                                                    subset ='validation')

Found 1228 images belonging to 14 classes.
Found 131 images belonging to 14 classes.


In [ ]:
train_data.image_shape

(224, 224, 3)

### 🔸Define Class, Layers, Optimizer and Loss Function

In [ ]:
class MyModel(tf.keras.Model):
  def __init__(self, number_of_classes):
    super().__init__()
    self.conv1 = Conv2D(16, (3, 3), activation='relu', input_shape = (width, height, 3))
    self.conv2 = Conv2D(32, (3, 3), activation='relu')
    self.conv3 = Conv2D(128, (3, 3), activation='relu')
    self.flatten = Flatten()
    self.maxpool = MaxPool2D()
    self.fc1 = Dense(128, activation='relu')
    self.fc2 = Dense(64, activation='relu')
    self.fc3 = Dense(number_of_classes, activation='softmax')

  def call(self, x):

    x = self.conv1(x)
    x = self.maxpool(x)
    x = self.conv2(x)
    x = self.conv3(x)
    x = self.flatten(x)
    x = self.fc1(x)
    x = self.fc2(x)
    output = self.fc3(x)

    return output

model = MyModel(14)

In [ ]:
loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer =  tf.keras.optimizers.Adam()

In [ ]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_acc = tf.keras.metrics.CategoricalAccuracy(name='train_acc')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_acc = tf.keras.metrics.CategoricalAccuracy(name='test_acc')

### 🔸wandb config

In [ ]:
configs = {
              "learning_rate": 0.001,
              "epochs": 25,
              "batch_size": 16,
              "log_step": 200,
              "val_log_step": 50
           }

run = wandb.init(project='Face_Recognition', config=configs)
config = wandb.config

### 🔸Training Process

In [ ]:
def train_step(images, y):
    with tf.GradientTape() as tape:
        y_pred = model(images, training = True)
        loss = tf.keras.losses.categorical_crossentropy(y, y_pred)

    #gradient calculate
    gradients = tape.gradient(loss, model.trainable_variables)

    #update
    optimizer.apply_gradients(zip(gradients,model.trainable_variables))

    train_loss(loss)
    train_acc(y, y_pred)


In [ ]:
def test_step(images, y):
  y_pred = model(images, training = False)
  loss = tf.keras.losses.categorical_crossentropy(y, y_pred)

  test_loss(loss)
  test_acc(y, y_pred)

In [ ]:
def train():
    epochs = 25

    for epoch in range(epochs):

        train_loss.reset_state()
        train_acc.reset_state()
        test_loss.reset_state()
        test_acc.reset_state()

        count_for_train = 0
        count_for_test = 0

        i_train = len(train_data)
        i_test = len(val_data)

        for images, labels in tqdm(train_data):
            train_step(images, labels)
            count_for_train += 1
            if count_for_train > i_train:
                break

        for images, labels in tqdm(val_data):
            test_step(images, labels)
            count_for_test += 1
            if count_for_test > i_test:
                break

        print('epoch:', epoch + 1)
        print('loss:', train_loss.result())
        print('accuracy:', train_acc.result())
        print('val loss:', test_loss.result())
        print('val accuracy:', test_acc.result())

        wandb.log({'epochs': epoch,
            'train_loss': np.mean(train_loss.result()),
            'train_acc': float(train_acc.result()), 
            'test_loss': np.mean(test_loss.result()),
            'test_acc': float(test_acc.result())
            })

### 🔸Train

In [ ]:
train()

100%|██████████| 9/9 [00:21<00:00,  2.34s/it]


epoch: 1
loss: tf.Tensor(3.1787233, shape=(), dtype=float32)
accuracy: tf.Tensor(0.32395497, shape=(), dtype=float32)
val loss: tf.Tensor(1.293384, shape=(), dtype=float32)
val accuracy: tf.Tensor(0.61904764, shape=(), dtype=float32)


100%|██████████| 9/9 [00:02<00:00,  3.31it/s]


epoch: 2
loss: tf.Tensor(0.7518353, shape=(), dtype=float32)
accuracy: tf.Tensor(0.7829582, shape=(), dtype=float32)
val loss: tf.Tensor(1.0150279, shape=(), dtype=float32)
val accuracy: tf.Tensor(0.6802721, shape=(), dtype=float32)


100%|██████████| 9/9 [00:02<00:00,  3.41it/s]


...

epoch: 23
loss: tf.Tensor(1.961369e-05, shape=(), dtype=float32)
accuracy: tf.Tensor(1.0, shape=(), dtype=float32)
val loss: tf.Tensor(0.78086627, shape=(), dtype=float32)
val accuracy: tf.Tensor(0.8503401, shape=(), dtype=float32)


100%|██████████| 9/9 [00:02<00:00,  3.29it/s]


epoch: 24
loss: tf.Tensor(1.6401726e-05, shape=(), dtype=float32)
accuracy: tf.Tensor(1.0, shape=(), dtype=float32)
val loss: tf.Tensor(0.78411317, shape=(), dtype=float32)
val accuracy: tf.Tensor(0.8639456, shape=(), dtype=float32)


100%|██████████| 9/9 [00:02<00:00,  3.45it/s]

epoch: 25
loss: tf.Tensor(1.722017e-05, shape=(), dtype=float32)
accuracy: tf.Tensor(1.0, shape=(), dtype=float32)
val loss: tf.Tensor(0.6756189, shape=(), dtype=float32)
val accuracy: tf.Tensor(0.8639456, shape=(), dtype=float32)


### 🔸Confusion Matrix

In [ ]:
Y_pred = model.predict(val_data)
y_pred = np.argmax(Y_pred, axis= 1)

print(confusion_matrix(val_data.classes, y_pred))
targets = set(val_data.class_indices.keys())
print('classification_report:')
print(classification_report(val_data.classes, y_pred, target_names= targets))

[[ 9  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  9  0  0  0  0  0  0  0  0  0  0  0  1]
 [ 0  1  7  0  0  0  0  1  0  0  0  0  0  1]
 [ 0  0  0 10  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  9  0  0  0  0  0  0  1  0  0]
 [ 0  0  0  0  0  8  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  1  6  0  0  0  0  0  0  2]
 [ 1  0  0  0  0  0  0  6  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  1  0  9  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  8  0  0  0  1]
 [ 0  1  0  0  0  0  0  0  0  0  6  0  0  2]
 [ 0  0  0  0  0  0  0  0  0  0  0 11  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  9  0]
 [ 0  1  0  0  0  1  0  0  0  1  0  0  0  7]]
classification_report:
                    precision    recall  f1-score   support

      Donald_Trump       0.90      1.00      0.95         9
      Lionel_Messi       0.75      0.90      0.82        10
    Michelle_Obama       1.00      0.70      0.82        10
   Queen_Elizabeth       1.00      1.00      1.00        10
       Barak_Obama       1.00      0.90      

### 🔸Save Weights

In [ ]:
model.save_weights('/content/drive/MyDrive/models/face_recognition_model.h5')

### 🔸Inference

In [ ]:
persons = [
           'مسی',
           'سوسانو',
           'میشل اوباما',
           'دونالد ترامپ',
           'اسکارلت جوهانسون',
           'انجلینا جولی',
           'باراک اوباما',
           'رهبر کره شمالی',
           'مرگان فریمن',
           'علی خامنه ای',
           'ملکه الیزابت',
           'بهنام بانی',
           'اما واتسون',
           'لیلا حاتمی',
           ]

In [ ]:
def process_and_predict(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = RetinaFace.extract_faces(image, align = True)
    image = image[0]
    image = image / 255.0
    image = cv2.resize(image, (224, 224))
    plt.imshow(image)
    image = tf.expand_dims(image, axis=0)

    y_pred = model(image, training = False)
    res = np.argmax(y_pred)
    return res

In [ ]:
print(persons[process_and_predict('leyla-hatami.jpeg')])

In [ ]:
print(persons[process_and_predict('trump.jpeg')])

In [ ]:
print(persons[process_and_predict('michelle-obama.jpeg')])